In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from functools import reduce
from collections import Iterable
    
# use for vertify 
from mlxtend.frequent_patterns import apriori, association_rules

/tmp/ipykernel_20944/643820373.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [2]:
real_dataset_path = "/home/sokhorn/sokhorn/dataSet/data/Online Retail.csv"
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv'

sample_dataset = pd.read_csv(
    real_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity'
    ])
sample_dataset


,InvoiceNo,StockCode,Quantity
0,536365,85123A,6
1,536365,71053,6
2,536365,84406B,8
3,536365,84029G,6
4,536365,84029E,6
...,...,...,...
541919,581598,22613,12
541920,581599,22899,6
541921,581600,23254,4
541922,581601,23255,4


In [3]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
# item_sets = item_sets.iloc[:, :10]
item_sets = item_sets.head(1000)

In [4]:
en_code = lambda x : 1 if x > 0  else 0
item_sets = item_sets.applymap(en_code)
item_sets

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Generating RC column for each tranctions**

In [5]:
def rc(item_np):
    np_hash = {}
    item_rc = []
    for item in item_np:
        key = " ".join(map(str, item))
        if np_hash.get(key):
            np_hash[key] += 1
        else:
            np_hash[key] = 1
    
    for item in np_hash:
        values =  list(map(int,  item.split()))
        values.append(np_hash[item])
        item_rc.append(values)
    return item_rc


In [6]:
rc_np = np.array(rc(item_sets.values))
df_rc =  pd.DataFrame(rc_np)
columns = list(item_sets.columns)
columns.append("RC")
df_rc.set_axis(columns, inplace=True  , axis=1 )
RC = df_rc['RC']
df_rc.drop(['RC'], axis=1, inplace=True)
df_rc

,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# reduce(function, input)
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s

In [8]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranctions[i])):
            count += 1
    return count

In [9]:
def join_set_item(set_of_its, order_column_name):
    C = []
    k = len(set_of_its[0])  # get K itmset size
    for i in range(len(set_of_its)):
        set_i = set_of_its[i]
        for j in range(i + 1, len(set_of_its)):
            set_j = set_of_its[j]
            if(k > 3):
                if(set_i[::k-1] == set_j[::k-1]):
                    it_out = join_two_itemsets(
                        set_i, set_j, order_column_name)
                    if(len(it_out)) > 0:
                        C.append(it_out)
            else:
                it_out = join_two_itemsets(
                    set_i, set_j, order_column_name)
                if(len(it_out)) > 0:
                    C.append(it_out)

    return C


In [10]:
'71053', '84029E', '84029G', '84406B','85123A'

('71053', '84029E', '84029G', '84406B', '85123A')

In [11]:
df_rc[['21730', '22752', ]]

,21730,22752
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
884,1,0
885,0,0
886,0,0
887,0,0


In [12]:
support_k_itemst(df_rc[['21730', '22752','84029G' ]].values)

9

In [13]:
df_rc[['21730', '22752']]

,21730,22752
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
884,1,0
885,0,0
886,0,0
887,0,0


**Fk-1 x Fk-1**

In [14]:
def apriori_gen(Lk, order):
    k = len(Lk)
    result_list = []
    for i in range(k):
        L1 = list(Lk[i])[::k-1]
        for j in range(i + 1, k):
            L2 = list(Lk[j])[::k-1]
            L1.sort(key=lambda x : order.index(x))
            L2.sort(key=lambda x : order.index(x))
            if L1 == L2:
                result_list.append(np.union1d(Lk[i], Lk[j]))
    return result_list

In [15]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x: order.index(x))
    it2.sort(key=lambda x: order.index(x))

    for i in range(len(it1) - 1):  # check befor the last one
        if it1[i] != it2[i]:
            return []

    if order.index(it1[-1]) < order.index(it2[-1]):
        return [it1] + [it2[-1]]
    return []


**Join Itemset**

In [16]:
def mergeKItemIntoOne(ab):
    result = []
    if(len(ab) != 0):
        if(len(ab[0]) == 1):
            return ab
        else:
            for item in ab:
                res = item[0] + item[1].split()
                result.append(res)
        return result


In [17]:
def get_frequent(itemesets, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []
    column_items = mergeKItemIntoOne(itemesets)

    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i] 
        result = []
        for i_item in item:
            if isinstance(i_item, list):
                for j in i_item:
                    result.append(j)
            else:
                result.append(i_item)

        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break
        
        if not discard_before:
            # print(f'item {mergeKItemIntoOne(item)}')
            count = support_k_itemst(df_rc[column_items[i]].values)
            if count >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard

In [18]:
def generate_next_itemset(L):
    k = L.shape[-1] + 1
    print(len(L))
    aa =  np.array(list(combinations(L, k)))
    col_name  =  np.hstack((aa[:, 0, :], aa[:, 1, :])) 
    return col_name

**Generating 1 itemse base on support count of row**

In [19]:
C = {}
L = {}
k_items = []
Discard = {}
itemset_size = 1
min_support = 1
Discard.update({itemset_size : []})

#### Generating 1 itemset

In [20]:
df_rc

,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
starat explore numpy

SyntaxError: invalid syntax (3340776440.py, line 1)

In [ ]:
# remove each column who support count are less than min_support for 1 itemsets
rc_1_itemset = df_rc.sum(axis=0)
cut_our_cols = rc_1_itemset.loc[lambda x: x < min_support].index # remove all column which sum of row are ness than user defind support threshold 
cut_our_cols
df_rc.drop(labels=cut_our_cols, axis=1, inplace=True) # cut our every itemset which are sum of each row are less than user defind support 
df_rc

,10002,10120,10123C,10124A,10124G,10125,10133,10135,11001,15034,...,90214V,90214Y,90214Z,AMAZONFEE,BANK CHARGES,C2,DCGS0076,DOT,M,POST
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
C.update({itemset_size: np.reshape(list(df_rc.columns), (-1, 1))})

In [ ]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})

In [ ]:
k = itemset_size + 1
while True:
    try:
      C.update({k: join_set_item(L[k - 1], list( df_rc.columns))})
      f, supp, new_discard = get_frequent(C[k], min_support, Discard)
      L.update({k: f})
      Discard.update({k: new_discard})
      support_count.update({k: supp})
      if(len(L[k]) == 0):
          break
      k += 1
    except:
      print("Hello error ")

Hello error 


In [ ]:
L

In [ ]:
# itemset = apriori(df_rc, min_support=min_support / len(item_sets), use_colnames=True)
# itemset

In [ ]:
# rules = association_rules(itemset, min_threshold=1)
# rules

In [ ]:
RC

In [ ]:
df_rc

#### Testing code 

In [ ]:
# support_k_itemst(df_rc[col_name[0]].values)

In [ ]:
# supp = support_k_itemst(df_rc[col_name].values)
# supp

In [ ]:
col_2_item =  np.array(list(product(L[3], L[1]))).T.reshape(-1, 2)
col_2_item

In [ ]:
L[1]

In [ ]:
join_set_item(L[1], list(df_rc.columns))

In [ ]:
df_rc

In [ ]:
# supp_2_itemset =  [support_k_itemst(df_rc[col].values) for col in col_2_item]
# supp_2_itemset

sort_by_index =  lambda cols : cols.index   
print(f"List before Sort {L[1]}")
l =  list(L[1]).sort(key =sort_by_index(list(df_rc.columns)))
print(f"List after Sort {l}")

### Debug code

In [ ]:
L

In [ ]:
Discard

In [ ]:
l_1 =  np.array(L[1])

In [ ]:
l_2 = [x for x in range(len(l_1))]
l_2

In [ ]:
L[3][:15]

In [ ]:
np.array(np.meshgrid(L[3], L[3]))

**Candedate Generation**

**F_k-1 x F_k-1**  <img src="images/fk-1xfk-1.png"  />

In [ ]:
k_itemset =  np.array(L[4])
k_itemset

In [ ]:
k_2 =  k_itemset[:,:4]
k_2

In [ ]:
unq,_ = np.unique(k_2, axis=0, return_inverse=True)
cnt = np.bincount(_)
unq = unq.view(k_2.dtype).reshape(-1, k_2.shape[1])

In [ ]:
unq =  unq.astype('object')
unq

In [ ]:
cnt

In [ ]:
uniqe_item =  np.column_stack((unq, cnt))
uniqe_item

In [ ]:
for i, j in combinations(unq[:6:], 2):
    print(i)

In [ ]:
unq[:6:][1]

In [ ]:
apriori_gen(unq)

In [ ]:
unq[10]

In [ ]:
unq[np.lexsort(unq[::3])]

In [ ]:
a = [ '84029E', '71053']
a.sort(key=lambda x : x.split())
a

### Start Explore numpy

In [68]:
product_names =  df_rc.columns[:3].values

In [69]:
product_names

array(['10002', '10080', '10120'], dtype=object)

In [70]:
product_names

array(['10002', '10080', '10120'], dtype=object)

In [71]:
def cartesian(*arrays):
    mesh = np.meshgrid(*arrays)  # standard numpy meshgrid
    dim = len(mesh)  # number of dimensions
    elements = mesh[0].size  # number of elements, any index will do
    flat = np.concatenate(mesh).ravel()  # flatten the whole meshgrid
    reshape = np.reshape(flat, (dim, elements)).T  # reshape and transpose
    return reshape

In [83]:
a = cartesian(product_names, product_names)

In [84]:
a

array([['10002', '10002'],
       ['10080', '10002'],
       ['10120', '10002'],
       ['10002', '10080'],
       ['10080', '10080'],
       ['10120', '10080'],
       ['10002', '10120'],
       ['10080', '10120'],
       ['10120', '10120']], dtype=object)

In [74]:
b = cartesian(a, a, a)
print(b)

[['10002' '10002' '10002']
 ['10002' '10002' '10002']
 ['10002' '10002' '10080']
 ...
 ['10120' '10120' '10120']
 ['10120' '10120' '10120']
 ['10120' '10120' '10120']]
